## Colab Issue


### Imports

In [1]:
import random

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

### Environment Information

In [2]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_dir = "~/.datasets"
seed = 42

random_seed(seed, torch.cuda.is_available())

print("Device:", device)
if torch.cuda.is_available():
  print("Device Name:", torch.cuda.get_device_name(0))
print("Data Dir:", data_dir)
print("Set Random Seed:", seed)

Device: cuda:0
Device Name: GeForce GTX 1660 Ti
Data Dir: ~/.datasets
Set Random Seed: 42


### Class to load the CIFAR10 dataset

In [3]:
class CIFAR10Data:
    def __init__(self):
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]
        )

        self.trainset = datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )
        self.testset = datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

        self.classes = (
            "plane",
            "car",
            "bird",
            "cat",
            "deer",
            "dog",
            "frog",
            "horse",
            "ship",
            "truck",
        )

    def get_cifar10_data(self):
        return self.trainset, self.testset, self.classes

    def get_cifar10_batch_loaders(self, batch_size=64):

        train_dataloader = DataLoader(
            self.trainset,
            shuffle=True,
            batch_size=batch_size,
            drop_last=True,
            num_workers=4,
        )
        test_dataloader = DataLoader(
            self.testset,
            shuffle=False,
            batch_size=batch_size,
            drop_last=True,
            num_workers=4,
        )

        return train_dataloader, test_dataloader


### The Trainer Class

In [4]:
class Trainer(object):
    """
    The Trainer class,
    which makes use of Torch's Module, Loss, Optimizer implementations
    """

    def __init__(
        self,
        model: nn.Module,
        loss: nn.modules.loss._Loss,
        optimizer: torch.optim.Optimizer,
    ):
        self._model = model
        self._loss = loss
        self._optimizer = optimizer

    def __str__(self):
        return f"""Trainer:\nArch:\n{self._model}\nLoss: {self._loss}\nOptimizer: {self._optimizer}"""

    def predict_classes(self, dataloader, retLabels=False):
        predY: [int] = []
        actualY: [int] = []
        for X, Y in dataloader:
            X, Y = X.to(device), Y
            X = X.reshape(dataloader.batch_size, -1).to(device)
            class_probabilities = self._model(X)
            class_predictions = torch.argmax(class_probabilities, dim=1)
            predY.extend(class_predictions.tolist())
            actualY.extend(Y.tolist())
        if retLabels:
            return predY, actualY
        else:
            return predY

    def fit(
        self,
        train_dataloader: torch.utils.data.DataLoader,
        test_dataloader: torch.utils.data.DataLoader,
        epochs=100,
        track_metrics=True,
        log=True,
        log_interval=10,
    ):
        train_losses = []
        test_losses = []
        train_accs = []
        test_accs = []

        for epoch in range(1, epochs + 1):
            self._model.train()
            train_loss = 0.0
            train_acc = 0.0
            for batch_id, (X, Y) in enumerate(train_dataloader):
                X, Y = X.to(device), Y.to(device)
                X = X.view(train_dataloader.batch_size, -1).to(device)
                self._optimizer.zero_grad()
                class_probabilities = self._model(X)
                loss = self._loss(class_probabilities, Y)

                if track_metrics:
                    train_loss += loss.item()
                    class_prediction = torch.argmax(class_probabilities, dim=1)
                    train_acc += (class_prediction == Y.data).sum().item()

                loss.backward()

                self._optimizer.step()

            if track_metrics:
                train_losses.append(train_loss)
                train_acc /= len(train_dataloader.dataset)
                train_accs.append(train_acc)

            test_loss, test_acc = self.validation(test_dataloader)
            test_accs.append(test_acc)
            if log and epoch % log_interval == 0:
                test_losses.append(test_loss)
                print(
                    "Epoch: {}\tTraining Loss: {:.4f}\tTraining Acc: {:.4f}\tTest Acc: {:.4f}\t".format(
                        epoch, train_loss, train_acc, test_acc
                    )
                )

        return train_accs, test_accs

    def validation(self, testloader):
        test_loss = 0
        accuracy = 0

        for testX, testY in testloader:
            testX, testY = testX.to(device), testY.to(device)
            testX = testX.reshape(testloader.batch_size, -1).to(device)

            predictions = self._model(testX)
            test_loss += self._loss(predictions, testY).item()

            argmax = torch.argmax(predictions, dim=1)
            accuracy += (argmax == testY.data).sum().item()

        return test_loss, accuracy / len(testloader.dataset)

### Initialize the Trainer class with a Model, Loss Function and Optimizer

In [5]:
cifar10_data = CIFAR10Data()


def run_model_trainer(learning_rate, batch_size, print_trainer_info=False, **kwargs):

    _model = nn.Sequential(
        nn.Linear(1024 * 3, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10),
        nn.Softmax(-1),
    )

    _trainer = Trainer(
        model=_model.to(device),
        loss=nn.CrossEntropyLoss(),
        optimizer=torch.optim.SGD(_model.parameters(), lr=learning_rate, momentum=0.9),
    )

    if print_trainer_info:
        print(_trainer)

    _train_dataloader, _test_dataloader = cifar10_data.get_cifar10_batch_loaders(
        batch_size=batch_size
    )

    _train_accs, _test_accs = _trainer.fit(
        _train_dataloader, _test_dataloader, **kwargs
    )

    return _trainer, (_train_dataloader, _test_dataloader), (_train_accs, _test_accs)

Files already downloaded and verified
Files already downloaded and verified


### Train the model on CIFAR10 and plot graphs

In [6]:
%%time

trainer, (train_dataloader, test_dataloader), _ = run_model_trainer(
    batch_size=64,
    learning_rate=1e-3,
    print_trainer_info=True,
    epochs=5,
    log_interval=1
)

Trainer:
Arch:
Sequential(
  (0): Linear(in_features=3072, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): Softmax(dim=-1)
)
Loss: CrossEntropyLoss()
Optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
Epoch: 1	Training Loss: 1796.3746	Training Acc: 0.1534	Test Acc: 0.1856	
Epoch: 2	Training Loss: 1790.5119	Training Acc: 0.1992	Test Acc: 0.1910	
Epoch: 3	Training Loss: 1775.8906	Training Acc: 0.1806	Test Acc: 0.1724	
Epoch: 4	Training Loss: 1753.6627	Training Acc: 0.1951	Test Acc: 0.2149	
Epoch: 5	Training Loss: 1733.7086	Training Acc: 0.2369	Test Acc: 0.2633	
CPU times: user 14.1 s, sys: 2.35 s, total: 16.5 s
Wall time: 19.4 s
